In [14]:
import pandas as pd
import sqlite3 as sq
from datetime import datetime

In [15]:
rs_stocks = pd.read_csv('../src/rs_stocks.csv')
rs_stocks

,Rank,Ticker,Sector,Industry,Exchange,Relative Strength,Percentile,1 Month Ago,3 Months Ago,6 Months Ago
0,1,AAOI,Technology,Semiconductors,NASDAQ,589.60,99,99,49,86
1,2,EDTX,Financial Services,Shell Companies,NASDAQ,534.06,99,99,72,75
2,3,EDTXU,Financial Services,Shell Companies,NASDAQ,497.75,99,99,59,55
3,4,RCRT,Industrials,Staffing & Employment Services,NASDAQ,453.53,99,5,3,2
4,5,CABA,Healthcare,Biotechnology,NASDAQ,401.17,99,99,99,99
...,...,...,...,...,...,...,...,...,...,...
6564,6565,AULT,Industrials,Aerospace & Defense,NYSE MKT,4.48,0,0,1,6
6565,6566,GMBL,Consumer Cyclical,Gambling,NASDAQ,3.83,0,0,4,0
6566,6567,SMX,Industrials,Specialty Business Services,NASDAQ,2.97,0,0,0,3
6567,6568,AVTX,Healthcare,Biotechnology,NASDAQ,2.67,0,0,35,3


In [16]:
mkt_cap_data = pd.read_csv('../src/TV.csv')
mkt_cap_data

,Ticker,Description,Market Capitalization,Upcoming Earnings Date
0,AAPL,Apple Inc.,2.947678e+12,2023-10-26
1,MSFT,Microsoft Corporation,2.439934e+12,2023-10-25
2,GOOG,Alphabet Inc.,1.734228e+12,2023-10-24
3,GOOGL,Alphabet Inc.,1.733458e+12,2023-10-24
4,AMZN,"Amazon.com, Inc.",1.416647e+12,2023-10-26
...,...,...,...,...
18164,RWX,SPDR DJ Wilshire Intl Real Estate,NaN,NaN
18165,JUST,Goldman Sachs JUST U.S. Large Cap Equity ETF,NaN,NaN
18166,INNO,Harbor Disruptive Innovation ETF,NaN,NaN
18167,IJT,iShares S&P SmallCap 600 Growth ETF,NaN,NaN


In [17]:
# Load the 'price_action' table from the 'market_data.db' database into a DataFrame named 'price_action'.
conn = sq.connect('../market_data.db')
price_action = pd.read_sql_query("SELECT * FROM price_action", conn)
conn.close()
price_action

,date,ticker,open,high,low,close,adjclose,volume
0,2020-01-02,AAOI,12.130000,12.530000,11.800000,12.500000,12.500000,885000.0
1,2020-01-03,AAOI,12.250000,12.325000,12.010000,12.120000,12.120000,498400.0
2,2020-01-06,AAOI,12.000000,12.200000,11.860000,12.150000,12.150000,361600.0
3,2020-01-07,AAOI,12.200000,12.465000,12.055000,12.430000,12.430000,330500.0
4,2020-01-08,AAOI,12.410000,12.820000,12.370000,12.700000,12.700000,520600.0
...,...,...,...,...,...,...,...,...
5319344,2023-08-24,RENEW,126.209999,126.919998,125.940002,126.019997,126.019997,0.0
5319345,2023-08-25,RENEW,125.260002,126.459999,125.169998,126.059998,126.059998,0.0
5319346,2023-08-28,RENEW,126.400002,127.019997,126.269997,126.709999,126.709999,0.0
5319347,2023-08-29,RENEW,126.760002,128.190002,126.760002,128.050003,128.050003,0.0


In [18]:
# Define the Minervini criteria with the new eliminatory criteria
def minervini_criteria_full(row):
    criteria = [
        row['Price'] > 5,
        row['Price'] < 300,
        row['Price'] > row['SMA50'],
        row['SMA50'] > row['SMA150'],
        row['SMA150'] > row['SMA200'],
        row['Price'] >= row['52Week-Low'] * 1.25,
        row['Price'] >= row['52Week-High'] * 0.85,
        row['Percentile'] >= 70,
        row['SMA200_1M_Trend'] == 1,
        row['Avg_Vol_90d'] > 100000,
        row['Avg_Vol_10d'] <= row['Avg_Vol_30d']

    ]
    return 'yes' if all(criteria) else 'no'

In [19]:
# Define the Minervini criteria with the new eliminatory criteria
def minervini_criteria_basic(row):
    criteria = [
        row['Price'] > 5,
        row['Price'] < 300,
        row['Price'] > row['SMA50'],
        row['SMA50'] > row['SMA150'],
        row['SMA150'] > row['SMA200'],
        row['Percentile'] >= 70,
        row['Avg_Vol_90d'] > 100000,

    ]
    return 'yes' if all(criteria) else 'no'

In [20]:
# Define the PowerTrend criteria
def power_trend(row, prev_power_trend):
    # Entry criteria
    if (
        row['lows_above_EMA21'] >= 10 and
        row['EMA21_above_SMA50'] >= 5 and
        row['SMA50_uptrend'] == 1 and
        row['close_above_open'] == 1
    ):
        return 'in'
    
    # Exit criteria
    if prev_power_trend == 'in' and (
        row['EMA21'] < row['SMA50'] or
        (row['close'] < row['high_last_week'] * 0.9 and row['close'] < row['SMA50'])
    ):
        return 'out'
    
    return prev_power_trend

In [21]:
screen_df =  pd.merge(price_action, mkt_cap_data, left_on='ticker', right_on='Ticker', how='left')
screen_df = pd.merge(screen_df, rs_stocks, left_on='Ticker', right_on='Ticker', how='left')
screen_df

,date,ticker,open,high,low,close,adjclose,volume,Ticker,Description,...,Upcoming Earnings Date,Rank,Sector,Industry,Exchange,Relative Strength,Percentile,1 Month Ago,3 Months Ago,6 Months Ago
0,2020-01-02,AAOI,12.130000,12.530000,11.800000,12.500000,12.500000,885000.0,AAOI,"Applied Optoelectronics, Inc.",...,2023-11-08,1,Technology,Semiconductors,NASDAQ,589.6,99,99,49,86
1,2020-01-03,AAOI,12.250000,12.325000,12.010000,12.120000,12.120000,498400.0,AAOI,"Applied Optoelectronics, Inc.",...,2023-11-08,1,Technology,Semiconductors,NASDAQ,589.6,99,99,49,86
2,2020-01-06,AAOI,12.000000,12.200000,11.860000,12.150000,12.150000,361600.0,AAOI,"Applied Optoelectronics, Inc.",...,2023-11-08,1,Technology,Semiconductors,NASDAQ,589.6,99,99,49,86
3,2020-01-07,AAOI,12.200000,12.465000,12.055000,12.430000,12.430000,330500.0,AAOI,"Applied Optoelectronics, Inc.",...,2023-11-08,1,Technology,Semiconductors,NASDAQ,589.6,99,99,49,86
4,2020-01-08,AAOI,12.410000,12.820000,12.370000,12.700000,12.700000,520600.0,AAOI,"Applied Optoelectronics, Inc.",...,2023-11-08,1,Technology,Semiconductors,NASDAQ,589.6,99,99,49,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5319344,2023-08-24,RENEW,126.209999,126.919998,125.940002,126.019997,126.019997,0.0,NaN,NaN,...,NaN,5940,Technology,Semiconductors,NASDAQ,54.0,9,57,92,7
5319345,2023-08-25,RENEW,125.260002,126.459999,125.169998,126.059998,126.059998,0.0,NaN,NaN,...,NaN,5940,Technology,Semiconductors,NASDAQ,54.0,9,57,92,7
5319346,2023-08-28,RENEW,126.400002,127.019997,126.269997,126.709999,126.709999,0.0,NaN,NaN,...,NaN,5940,Technology,Semiconductors,NASDAQ,54.0,9,57,92,7
5319347,2023-08-29,RENEW,126.760002,128.190002,126.760002,128.050003,128.050003,0.0,NaN,NaN,...,NaN,5940,Technology,Semiconductors,NASDAQ,54.0,9,57,92,7


In [15]:
screen_df

,date,ticker,open,high,low,close,adjclose,volume,Ticker,Description,...,Upcoming Earnings Date,Rank,Sector,Industry,Exchange,Relative Strength,Percentile,1 Month Ago,3 Months Ago,6 Months Ago
0,2023-08-22,A,117.849998,118.610001,117.330002,117.639999,117.639999,1422300.0,A,"Agilent Technologies, Inc.",...,2023-11-20,4043.0,Healthcare,Diagnostics & Research,NYSE,85.13,38.0,29.0,33.0,57.0
1,2023-08-23,A,118.510002,119.150002,117.709999,119.129997,119.129997,1289300.0,A,"Agilent Technologies, Inc.",...,2023-11-20,4043.0,Healthcare,Diagnostics & Research,NYSE,85.13,38.0,29.0,33.0,57.0
2,2023-08-24,A,119.360001,120.779999,118.580002,119.489998,119.489998,2045900.0,A,"Agilent Technologies, Inc.",...,2023-11-20,4043.0,Healthcare,Diagnostics & Research,NYSE,85.13,38.0,29.0,33.0,57.0
3,2023-08-25,A,119.800003,120.820000,118.519997,119.680000,119.680000,1273300.0,A,"Agilent Technologies, Inc.",...,2023-11-20,4043.0,Healthcare,Diagnostics & Research,NYSE,85.13,38.0,29.0,33.0,57.0
4,2023-08-22,AA,28.910000,29.090000,28.180000,28.780001,28.780001,4594300.0,AA,Alcoa Corporation,...,2023-10-12,5614.0,Basic Materials,Aluminum,NYSE,63.88,14.0,18.0,13.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6893175,2023-08-29,ZYNE,1.320000,1.340000,1.320000,1.320000,1.320000,282500.0,ZYNE,"Zynerba Pharmaceuticals, Inc.",...,2023-11-13,18.0,Healthcare,Drug Manufacturers—Specialty & Generic,NASDAQ,256.61,99.0,10.0,8.0,5.0
6893176,2023-08-30,ZYNE,1.320000,1.330000,1.320000,1.320000,1.320000,499700.0,ZYNE,"Zynerba Pharmaceuticals, Inc.",...,2023-11-13,18.0,Healthcare,Drug Manufacturers—Specialty & Generic,NASDAQ,256.61,99.0,10.0,8.0,5.0
6893177,2023-08-28,ZYXI,7.700000,7.770000,7.570000,7.660000,7.660000,200700.0,ZYXI,"Zynex, Inc.",...,2023-10-31,5541.0,Healthcare,Medical Distribution,NASDAQ,66.02,15.0,10.0,57.0,77.0
6893178,2023-08-29,ZYXI,7.650000,7.740000,7.550000,7.710000,7.710000,247900.0,ZYXI,"Zynex, Inc.",...,2023-10-31,5541.0,Healthcare,Medical Distribution,NASDAQ,66.02,15.0,10.0,57.0,77.0


In [23]:
# Add the 'Price' column representing the latest close for each stock
screen_df['Price'] = screen_df.groupby('Ticker')['close'].transform('last')

# Calculate the 50-day Simple Moving Average (SMA50) for each stock.
screen_df['SMA50'] = screen_df.groupby('Ticker')['close'].transform(lambda x: x.rolling(window=50).mean())

# Calculate the 21-day Exponential Moving Average (EMA21) for each stock.
screen_df['EMA21'] = screen_df.groupby('Ticker')['close'].transform(lambda x: x.ewm(span=21, adjust=False).mean())

# Calculate the 150-day Simple Moving Average (SMA150) for each stock.
screen_df['SMA150'] = screen_df.groupby('Ticker')['close'].transform(lambda x: x.rolling(window=150).mean())

# Calculate the 200-day Simple Moving Average (SMA200) for each stock.
screen_df['SMA200'] = screen_df.groupby('Ticker')['close'].transform(lambda x: x.rolling(window=200).mean())

# Calculate the 90-day Average Volume for each stock.
screen_df['Avg_Vol_90d'] = screen_df.groupby('Ticker')['volume'].transform(lambda x: x.rolling(window=90).mean())

# Calculate the 30-day Average Volume for each stock.
screen_df['Avg_Vol_30d'] = screen_df.groupby('Ticker')['volume'].transform(lambda x: x.rolling(window=30).mean())

# Calculate the 10-day Average Volume for each stock.
screen_df['Avg_Vol_10d'] = screen_df.groupby('Ticker')['volume'].transform(lambda x: x.rolling(window=10).mean())

# Calculate the 52-week high (highest closing price in the last 252 trading days) for each stock.
screen_df['52Week-High'] = screen_df.groupby('Ticker')['close'].transform(lambda x: x.rolling(window=252).max())

# Calculate the 52-week low (lowest closing price in the last 252 trading days) for each stock.
screen_df['52Week-Low'] = screen_df.groupby('Ticker')['close'].transform(lambda x: x.rolling(window=252).min())

# Calculate the price performance over the last 252 trading days
screen_df['12M Return'] = screen_df.groupby('Ticker')['close'].pct_change(periods=252)

# Calculate the highest high and lowest low over the last 5 daysb g
screen_df['10D_high'] = screen_df.groupby('Ticker')['high'].transform(lambda x: x.rolling(window=10).max())
screen_df['10D_low'] = screen_df.groupby('Ticker')['low'].transform(lambda x: x.rolling(window=10).min())
# Calculate the range as a percentage of the highest highb g
screen_df['10D_range_pct'] = (screen_df['10D_high'] - screen_df['10D_low']) / screen_df['10D_high'] * 100

# Identify low-risk entries
screen_df['Low-Risk_Entry'] = (screen_df['10D_range_pct'] < 8).astype(int)

# Calculate the SMA200 trend over the last month
screen_df['SMA200_1M_Trend'] = screen_df.groupby('Ticker')['SMA200'].transform(lambda x: x.diff(periods=21)).gt(0).astype(int)

# Calculate criteria for PowerTrend
screen_df['lows_above_EMA21'] = screen_df.groupby('Ticker').apply(lambda x: (x['low'] > x['EMA21']).rolling(window=10).sum()).reset_index(level=0, drop=True)
screen_df['EMA21_above_SMA50'] = screen_df.groupby('Ticker').apply(lambda x: (x['EMA21'] > x['SMA50']).rolling(window=5).sum()).reset_index(level=0, drop=True)
screen_df['SMA50_uptrend'] = screen_df.groupby('Ticker')['SMA50'].transform(lambda x: x.diff().rolling(window=21).sum()).gt(0).astype(int)
screen_df['close_above_open'] = (screen_df['close'] > screen_df['open']).astype(int)
screen_df['high_last_week'] = screen_df.groupby('Ticker')['high'].transform(lambda x: x.rolling(window=5).max())

# Apply the Minervini criteria to each row
screen_df['Minervini_basic'] = screen_df.apply(minervini_criteria_basic, axis=1)
screen_df['Minervini_full'] = screen_df.apply(minervini_criteria_full, axis=1)

# Apply the PowerTrend criteria to each row
prev_power_trend = 'out'
power_trends = []

for _, row in screen_df.iterrows():
    current_power_trend = power_trend(row, prev_power_trend)
    power_trends.append(current_power_trend)
    prev_power_trend = current_power_trend
    
# Apply the Minervini criteria to each row
screen_df['PowerTrend'] = power_trends

# Create the 'screen_df' DataFrame with the calculated metrics.
metrics = ['Ticker', 'EMA21', 'SMA50', 'SMA150', 'SMA200', 'Avg_Vol_10d', 'Avg_Vol_30d', 'Avg_Vol_90d', '52Week-High', 
           '52Week-Low', 'Minervini_basic', 'Minervini_full', 'PowerTrend', 'Low-Risk_Entry', 'Market Capitalization',
           'Upcoming Earnings Date', 'Industry', 'Percentile', '1 Month Ago', '3 Months Ago', '6 Months Ago']
screen_df = screen_df.drop_duplicates(subset='Ticker', keep='last')[metrics]
screen_df

,Ticker,EMA21,SMA50,SMA150,SMA200,Avg_Vol_10d,Avg_Vol_30d,Avg_Vol_90d,52Week-High,52Week-Low,...,Minervini_full,PowerTrend,Low-Risk_Entry,Market Capitalization,Upcoming Earnings Date,Industry,Percentile,1 Month Ago,3 Months Ago,6 Months Ago
921,AAOI,12.823793,9.559300,4.796267,4.133850,3443830.0,4.922603e+06,2.982519e+06,15.480000,1.6400,...,yes,in,0,4.621735e+08,2023-11-08,Semiconductors,99,99,49,86
3195,RCRT,0.674549,0.333364,0.310218,0.342089,5179070.0,2.161693e+06,2.531102e+06,1.840000,0.1419,...,no,out,0,2.394363e+06,2023-11-13,Staffing & Employment Services,99,5,3,2
4117,CABA,13.432683,12.869400,10.878133,9.913300,319220.0,3.405033e+05,4.571067e+05,14.560000,0.6290,...,yes,out,0,5.560763e+08,2023-11-14,Biotechnology,99,99,99,99
5039,CVNA,42.202502,38.378400,20.167200,16.683400,9341160.0,1.499711e+07,2.554712e+07,55.799999,3.7200,...,yes,in,0,9.403993e+09,2023-11-02,Internet Retail,99,99,91,61
5205,ENLT,17.072340,17.844600,16.437200,NaN,23400.0,3.806333e+04,5.759778e+04,NaN,NaN,...,no,out,1,1.902217e+09,NaN,Utilities - Renewable,99,99,99,99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5316149,AULT,1.968874,3.573600,19.329267,24.156950,391120.0,6.620467e+05,5.752539e+05,90.000000,1.0400,...,no,out,0,5.997268e+06,2023-11-22,Aerospace & Defense,0,0,1,6
5317071,GMBL,0.251263,0.617240,2.519547,4.382660,17969140.0,1.192034e+07,4.055897e+06,48.000000,0.1220,...,no,out,0,4.985465e+06,2023-10-12,Gambling,0,0,4,0
5317504,SMX,2.332217,3.136780,50.109407,93.594605,269810.9,2.489654e+05,2.454848e+05,321.200012,1.4600,...,no,out,0,3.748505e+06,NaN,Specialty Business Services,0,0,0,3
5318426,AVTX,0.176385,0.477080,2.067353,2.852570,16213340.0,7.559773e+06,3.375800e+06,6.900000,0.0890,...,no,out,0,2.115489e+06,2023-11-15,Biotechnology,0,0,35,3


In [25]:
today = datetime.now().strftime('%Y-%m-%d')
nome_arquivo = f'../output/screen_{today}.xlsx'
screen_df.to_excel(nome_arquivo, engine='xlsxwriter')

In [ ]:
# Get the current date
current_date = datetime.now().strftime('%d-%m-%Y')
table_name = f"screen-{current_date}"

# Connect to the SQLite database
conn = sq.connect('../market_data.db')

# Save the screen_df to the database with the table name based on the current date
screen_df.to_sql(table_name, conn, if_exists='replace', index=False)

# Close the database connection
conn.close()